# 딥러닝을 적용한 Q-learning
- 이는 표형식 표현을 사용해 표현을 하면 이미지와 같은 데이터를 사용할 때 행의 수(상태 == s_t)가 증가하게 되어 학습을 위해서는 많은 episode가 필요하기에 학습시키기가 어렵다
- 따라서 딥러닝을 적용하는 것은 "상태 변수 수가 많아지면 표형식 표현으로는 강화학습이 어렵다"
- 이를 해결하고자 행동가치 함수를 표형식으로 나타내는 대신 층 수가 많은 신경망으로 바꾸어 나타낸다.

# DQN(Deep Q Learning Network) 구현 (CartPole)

In [60]:
import numpy as np
import matplotlib.pyplot as plt
import gym
%matplotlib inline

In [61]:
# 애니메이션을 만드는 함수
# 참고 URL http://nbviewer.jupyter.org/github/patrickmineault/xcorr-notebooks/blob/master/Render%20OpenAI%20gym%20as%20GIF.ipynb
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0),
               dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames),
                                   interval=50)

    #anim.save('movie_cartpole.mp4')  # 주석 추가 : 애니메이션을 저장하는 부분
    display(display_animation(anim, default_mode='loop'))

In [62]:
# 이 코드에서는 namedtuple을 사용한다.
# 이를 사용하면 key-value의 쌍 형태로 값을 저장할 수 있다.
# 그리고 키를 필드명으로 값에 접근할 수 있어 편리하다

# from collections import namedtuple
# Tr = namedtuple('tr', ('name_a', 'value_b'))
# Tr_object = Tr('이름A', 100)
# print(Tr_object)
# print(Tr_object.value_b)
# tr(name_a='이름A', value_b=100)
# 100

from collections import namedtuple
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

In [63]:
ENV = 'CartPole-v0'
GAMMA = 0.99 # 시간 할인율
MAX_STEPS = 200 # 1 에피소드 당 최대 단계수
NUM_EPISODES = 500 # 최대 에피소드 수

In [64]:
# Transition을 저장하기 위한 메모리 클래스
class ReplayMemory:
    def __init__(self, CAPACITY):
        self.capacity = CAPACITY # 메모리의 최대 저장 건수
        self.memory = [] # 실제 transition을 저장할 변수
        self.index = 0 # 저장 위치를 가리킬 인덱스 변수
    
    def push(self, state, action, next_state, reward):
        # transition = (state, action, next_state, reward)을 메모리에 저장
        if len(self.memory) < self.capacity:
            self.memory.append(None) # 메모리가 가득 차지 않은 경우
        self.memory[self.index] = Transition(state, action, next_state, reward) # key-value 쌍으로 값을 저장
        self.index = (self.index + 1) % self.capacity
        
    def sample(self, batch_size):
        # batch_size만큼 무작위로 저자된 transition을 추출
        return random.sample(self.memory, batch_size) # sample함수는 첫번째 인자에 있는 값중에서 두번째 인자의 개수만큼 무직위로 추출하는 함수
    
    def __len__(self):
        return len(self.memory)

In [65]:
# Agent의 두뇌 역할을 하는 클래스. DQN을 실제 수행한다
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000

In [66]:
class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        self.memory = ReplayMemory(CAPACITY) # 메모리 객체 생성
        
        # 신경망 구성
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))
        print(self.model)
        
        # 최적화 기법 선택
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
    
    def replay(self):
        '''Experience Replay로 신경망의 결합 가중치 학습'''
        # ----------------------------------
        # 1. 저장된 transition 수 확인
        # ----------------------------------
        if len(self.memory) < BATCH_SIZE:
            return # 저장된 transition수가 미니 배치 수 보다 작으면 아무것도 하지 않음
        # ---------------------------------
        # 2. 미니 배치 생성
        # ---------------------------------
        transitions = self.memory.sample(BATCH_SIZE)
        
        # 각 변수를 미니배치에 맞는 형태로 변형
        # transitions는 (state, action, next_state, reward) * batch_size형태로 되어 있는 것을 분배법칙으로 나눠야 함
        # zip(*variable)의 경우 list로 된 variable을 unpacking하는 것이다. ==> [(1, a), (2, b)]라 가정하고 unpacking을 하면 (1, 2), (a, b)가 됨
        batch = Transition(*zip(*transitions))

        # 상태, 행동 보상, non_final 상태로 된 미니 배치를 나타내는 Variable생성
        # cat() 은 차원을 기준으로 합친다.
        state_batch = torch.cat(batch.state) # shape : (batch_size, state)
        action_batch = torch.cat(batch.action) # shape : (batch_size, action)
        reward_batch = torch.cat(batch.reward) # shape : (batch_size, reward)
        non_final_next_states = torch.cat([s for s in batch.next_state if s is not None]) # 게임이 끝난 경우는 추가적인 학습을 하지 않음
        
        # ----------------------------------
        # 3. 정답 신호로 사용할 Q(s_t, a_t)를 계산 ==> s_t상태에서 a_t를 행동했을 취했을 때에 대한 행동가치 함수 값을 구함
        # ----------------------------------
        self.model.eval() # 추론 모드로 변경
        # 이는 모델의 결과인 (batch_size, 2) 형태에서 dimension = 1을 기준으로 action_batch에서 선택한 행동인 index를 선택하여 변수에 저장함
        state_action_values = self.model(state_batch).gather(1, action_batch)
        
        # max{Q(s_t+1, a)}를 구함. 이때 다음 상태가 존재하는지 주의해야 함
        # cartpole이 done상태가 아니고, next_state가 존재하는지 확인하는 인덱스 마스크를 만듬
        non_final_mask = torch.BoolTensor(tuple(map(lambda s: s is not None, batch.next_state))) # map(함수, 리스트), lambda 인자 : 표현식

        # 먼저 전체를 0으로 초기화
        next_state_values = torch.zeros(BATCH_SIZE)
        
        # 다음 상태가 있는 인덱스에 대한 최대 Q값을 구한다.
        # 열방향 최댓값이 되는 [값, 인덱스]를 구하고 Q값을 출력한 다음 detach()로 값을 꺼내온다
        # pytorch에서 detach() 메서드를 사용하면 이 변수가 갖고 있던 지금까지의 계산 이력을 잃어버리고 오차 역전파를 할 때도 미분을 계산하지 않게 된다.
        next_state_values[non_final_mask] = self.model(non_final_next_states).max(axis = 1)[0].detach()
        
        # Q(s_t, a_t) 값을 Q-learning으로 계산
        expected_state_action_values = reward_batch + GAMMA * next_state_values
        
        # ----------------------------------
        # 4. 결합 가중치 수정
        # ----------------------------------
        self.model.train() # 학습 모드로 변경
        # 손실 함수를 계산 (smooth_l1_loss 는 Huber함수) # size를 [minibatch] ==> [minibatch * 1]로 변경
        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1)) 
        
        # 결합 가중치 수정
        self.optimizer.zero_grad() # 경사를 초기화
        loss.backward() # 역전파 계산
        self.optimizer.step() # 결합 가중치 수정
        
    def decide_action(self, state, episode):
        epsilon = 0.5 * (1 / (episode + 1))
        if epsilon <= np.random.uniform(0, 1):
            self.model.eval()
            with torch.no_grad():
                action = self.model(state).max(1)[1].reshape(1, 1) # state를 실행했을 때의 열의 기준으로 얻는 최댓값에서 인덱스를 얻어서 1 * 1형태로 변환
        else:
            action = torch.LongTensor([[random.randrange(self.num_actions)]])
            
        return action

In [67]:
# CartPole 태스크의 에이전트 클래스. 봉 달린 수레 자체라고 보면 된다
class Agent:
    def __init__(self, num_states, num_actions):
        '''태스크의 상태 및 행동의 가짓수를 설정'''
        self.brain = Brain(num_states, num_actions)  # 에이전트의 행동을 결정할 두뇌 역할 객체를 생성

    def update_q_function(self):
        '''Q함수를 수정'''
        self.brain.replay()

    def get_action(self, state, episode):
        '''행동을 결정'''
        action = self.brain.decide_action(state, episode)
        return action

    def memorize(self, state, action, state_next, reward):
        '''memory 객체에 state, action, state_next, reward 내용을 저장'''
        self.brain.memory.push(state, action, state_next, reward)

In [68]:
# CartPole을 실행하는 환경 역할을 하는 클래스
class Environment:
    def __init__(self):
        self.env = gym.make(ENV) # 태스크를 설정
        num_states = self.env.observation_space.shape[0] # 상태 변수를 받아옴
        num_actions = self.env.action_space.n # 태스크의 행동 가짓수를 받아옴
        self.agent = Agent(num_states, num_actions)
        
    def run(self):
        episode_10_list = np.zeros(10) # 최근 10에피소드 동안 버틴 단계수를 저장함
        complete_episode = 0 # 현재까지 195단계를 버틴 에피소드 수
        episode_final = False
        frames = []
        
        for episode in range(NUM_EPISODES):
            observation = self.env.reset() # 환경 초기화
            state = observation
            state = torch.from_numpy(state).type(torch.FloatTensor) # Numpy를 Pytorch Tensor로 변환
            state = torch.unsqueeze(state, 0) # size 4 ==> size 1*4
            
            for step in range(MAX_STEPS):
                if episode_final is True:
                    frames.append(self.env.render(mode='rgb_array'))
                action = self.agent.get_action(state, episode)
                
                # 행동을 실행해 다음 상태와 done 플래그 값을 결정
                observation_next, _, done, _ = self.env.step(action.item())
                
                if done: # 단계 수가 200을 넘었거나 봉이 일정 각도 이상 기울면 done이 True가 됨
                    state_next = None # 다음 상태가 없으므로
                    episode_10_list = np.hstack((episode_10_list[1:], step+1))
                    
                    if step < 195:
                        reward = torch.FloatTensor([-1.0])
                        complete_episode = 0
                    else:
                        reward = torch.FloatTensor([1.0])
                        complete_episode += 1
                else:
                    reward = torch.FloatTensor([0.0])
                    state_next = observation_next
                    state_next = torch.from_numpy(state_next).type(torch.FloatTensor)
                    state_next = torch.unsqueeze(state_next, 0) # size 4 ==> size 1 * 4
                
                self.agent.memorize(state, action, state_next, reward) # 메모리에 경험을 저장
                
                # Exprience Replay로 Q함수를 수정
                self.agent.update_q_function()
                # 관측 결과를 업데이트
                state = state_next
                
                if done:
                    print(f"{episode} Episode : Finished after {(step + 1)} steps : 최근 10 에피소드의 평균 단계 수 : {(episode_10_list.mean()):.1f}")
                    break
                if episode_final is True:
                    break
                    
                if complete_episode >= 10:
                    print('10 episode 연속 성공')
                    episode_final = True

In [69]:
cartpole_env = Environment()
cartpole_env.run()

Sequential(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)
0 Episode : Finished after 12 steps : 최근 10 에피소드의 평균 단계 수 : 1.2
1 Episode : Finished after 12 steps : 최근 10 에피소드의 평균 단계 수 : 2.4
2 Episode : Finished after 9 steps : 최근 10 에피소드의 평균 단계 수 : 3.3
3 Episode : Finished after 9 steps : 최근 10 에피소드의 평균 단계 수 : 4.2
4 Episode : Finished after 10 steps : 최근 10 에피소드의 평균 단계 수 : 5.2
5 Episode : Finished after 11 steps : 최근 10 에피소드의 평균 단계 수 : 6.3
6 Episode : Finished after 11 steps : 최근 10 에피소드의 평균 단계 수 : 7.4
7 Episode : Finished after 9 steps : 최근 10 에피소드의 평균 단계 수 : 8.3
8 Episode : Finished after 8 steps : 최근 10 에피소드의 평균 단계 수 : 9.1
9 Episode : Finished after 10 steps : 최근 10 에피소드의 평균 단계 수 : 10.1
10 Episode : Finished after 11 steps : 최근 10 에피소드의 평균 단계 수 : 10.0
11 Episode : Finished after 9 steps : 최근 10 에피소드의 평균 단계 수 : 9.7
12 Ep

123 Episode : Finished after 60 steps : 최근 10 에피소드의 평균 단계 수 : 50.7
124 Episode : Finished after 33 steps : 최근 10 에피소드의 평균 단계 수 : 50.2
125 Episode : Finished after 52 steps : 최근 10 에피소드의 평균 단계 수 : 49.7
126 Episode : Finished after 54 steps : 최근 10 에피소드의 평균 단계 수 : 48.9
127 Episode : Finished after 45 steps : 최근 10 에피소드의 평균 단계 수 : 48.8
128 Episode : Finished after 32 steps : 최근 10 에피소드의 평균 단계 수 : 48.0
129 Episode : Finished after 43 steps : 최근 10 에피소드의 평균 단계 수 : 48.8
130 Episode : Finished after 51 steps : 최근 10 에피소드의 평균 단계 수 : 50.4
131 Episode : Finished after 68 steps : 최근 10 에피소드의 평균 단계 수 : 48.7
132 Episode : Finished after 56 steps : 최근 10 에피소드의 평균 단계 수 : 49.4
133 Episode : Finished after 76 steps : 최근 10 에피소드의 평균 단계 수 : 51.0
134 Episode : Finished after 65 steps : 최근 10 에피소드의 평균 단계 수 : 54.2
135 Episode : Finished after 39 steps : 최근 10 에피소드의 평균 단계 수 : 52.9
136 Episode : Finished after 112 steps : 최근 10 에피소드의 평균 단계 수 : 58.7
137 Episode : Finished after 86 steps : 최근 10 에피소드의 평균 단계 수 :